In [1]:
#create a spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Trabalho prático").getOrCreate()

#print the spark version
print(spark.version)


3.5.0


### Lendo o arquivo de metadados do filme

In [8]:
df_titles = spark.read.csv('/home/jovyan/data/title_basics.tsv', \
                            header=True, \
                            inferSchema=True, \
                            sep='\t')

#print the dataframe schema
df_titles.printSchema()


root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [14]:
#show me the dataframe df_titles
df_titles.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [15]:
df_ratings = spark.read.csv('/home/jovyan/data/title_ratings.tsv', \
                            header=True, sep='\t')
df_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
+---------+-------------+--------+
only showing top 5 rows



In [19]:
#join the dataframes df_titles and df_ratings by key tconst
movies = df_titles.join(df_ratings, on='tconst', how='inner')

#show me df_films
movies.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+-------------+--------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|Documentary,Short|          5.5|    1965|
|tt0000015|    short| Autour d'une cabine| Autour d'une cabine|      0|     1894|     \N|             2|  Animation,Short|          6.2|     947|
|tt0000019|    short|    The Clown Barber|    The Clown Barber|      0|     1898|     \N|            \N|     Comedy,Short|          5.3|      28|
|tt0000051|    short|The Bohemian Enca...|Campement de bohé...|      0|     1896|     \N|            \N|Documentary,Short|  

In [ ]:
#group by for avarageRating and num Votes Desc
def get_top_movies(movies):
    movies = movies.groupby(['primaryTitle', 'year']).agg({'numVotes': 'sum', 'averageRating': 'mean'}).reset_index()
    movies = movies.sort_values(['numVotes', 'averageRating'], ascending=[False, False])
    return movies.head(10)